In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
#from tqdm.auto import tqdm
from tqdm import tqdm
plt.ioff();
try:
    maindir
except NameError:
    maindir = os.path.dirname(os.getcwd())
for module in tqdm(os.listdir(maindir + '\\testing_modules')):
    %run -i "{maindir}\\testing_modules\\{module}"
base = os.path.dirname(os.path.dirname(maindir)) + '\\'
plt.ion();
np.set_printoptions(suppress=True)
plt.close('All')

100%|██████████| 22/22 [02:31<00:00,  6.89s/it]


In [2]:
%matplotlib qt
import skimage.io as io
import xarray as xr
from collections import OrderedDict
import dask
from tqdm.dask import TqdmCallback
from sklearn.metrics.pairwise import euclidean_distances
from matplotlib import cm

In [4]:
scanid = 149566
filedir = '''D:\\Musterman_postdoc\\20231109_Musterman\\'''
filedir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Data\\Condensed Datasets\\'''
h5_filename = f'scan{scanid}_xrd.h5'

test = XRDMap.from_h5(h5_filename, wd=filedir)

Loading data from h5 file...
Loading most recent images (raw_images)...done!
Loading reciprocal positions...done!
Loading saved phases...done!
Loading reflection spots...done!
XRD Map loaded!


In [106]:
scanid = 149248
#filedir = '''D:\\Musterman_postdoc\\20231109_Musterman\\'''
filedir = '''D:\\Musterman_postdoc\\20231030_Musterman\\'''
filedir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Data\\Condensed Datasets\\'''
h5_filename = f'scan{scanid}_xrd.h5'

test = XRDMap.from_h5(h5_filename, wd=filedir)

Loading data from h5 file...
Loading most recent images (raw_images)...

In [ ]:
scanid = 149248
filedir = '''D:\\Musterman_postdoc\\20231030_Musterman\\'''
filedir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Data\\Condensed Datasets\\'''
map_filename = f'scan{scanid}_dexela_xrd.tif'
h5_filename = f'scan{scanid}_xrd.h5'

test = XRDMap.from_image_stack(map_filename, wd=filedir, energy=15, scanid=scanid)
dark_field = io.imread(f'{filedir}dexela_df_avg.tif')
test.set_calibration('scan149256_dexela_calibration.poni')

In [3]:
scanid = 149566
filedir = '''D:\\Musterman_postdoc\\20231109_Musterman\\'''
filedir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Data\\Condensed Datasets\\'''
map_filename = f'scan{scanid}_dexela_xrd.tif'
h5_filename = f'scan{scanid}_xrd.h5'

test = XRDMap.from_image_stack(map_filename, wd=filedir, energy=15, scanid=scanid)
test.set_calibration('scan149562_dexela_calibration.poni')

Loading images...done!
Writing images to h5...done!


In [319]:
filedir = '''D:\\Musterman_postdoc\\20240125_Musterman\\'''
scanid = 152636

test = XRDMap.from_image_stack('scan152636_dexela_energy_rc.tif', wd=filedir + 'amended\\', energy=12, scanid=scanid, save_hdf=False)
test.set_calibration('scan152648_calibration.poni', filedir=filedir)

Loading images...done!
Calibration performed under different settings. Adjusting calibration.


In [689]:
def estimate_depth_error(xrdmap, thickness):

    # Check input type 
    try:
        bounds = list(iter(thickness))
    except TypeError:
        bounds = [thickness, -thickness]
 
    # Retrieve pixel coordinates
    zyx = xrdmap.ai.position_array()
    # The beamline coordinates flip the x direction
    xyz = np.asarray([zyx[:, :, 2], zyx[:, :, 1], zyx[:, :, 0]])

    # Fit detector to a plane equation. Cannot handle non planar detectors...
    popt, _ = curve_fit(plane_eq, [xyz[0].ravel(), xyz[1].ravel()], xyz[2].ravel())

    tth_err = []
    chi_err = []
    for bound in bounds:
        if bound == 0:
            tth_err.append(xrdmap.tth_arr)
            chi_err.append(xrdmap.chi_arr)
            continue

        # Intersect offset vector with detector plane
        x, y, z = _find_intersection(0, 0, bound, xyz, popt)

        # Determine the new polar coordinates from this intersection
        # Must be arctan2 functions
        #chi_err.append(np.degrees(np.arctan2(y, x)))
        #tth_err.append(np.degrees(np.arctan2(np.sqrt(x**2 + y**2), z)))

        chi_err.append(np.degrees(np.arccos(x / np.sqrt(x**2 + y**2))))
        tth_err.append(np.degrees(np.arccos(z / np.sqrt(x**2 + y**2 + z**2))))


    return np.asarray(tth_err), np.asarray(chi_err)

In [690]:
tth_err, chi_err = estimate_depth_error(test, [0, 500e-6])

In [691]:
d = tth_2_d(tth_err, wavelength=test.wavelength)

In [692]:
dd = np.abs(d[0] - d[1])

In [694]:
chi1, chi2 = chi_err.copy()
chi1[chi1 < 0] += 360
chi2[chi2 < 0] += 360

dd = np.abs(chi1 - chi2)

In [696]:
dd = np.abs(tth_err[0] - tth_err[1])

In [699]:
test.plot_image(chi_err[1])

In [675]:
# Plot detector position
zyx = test.ai.position_array()

xyz = np.asarray([zyx[:, :, 2], zyx[:, :, 1], zyx[:, :, 0]])

In [676]:
popt, pcov = curve_fit(plane_eq, [xyz[0].ravel(), xyz[1].ravel()], xyz[2].ravel())
print(compute_r_squared(xyz[-1].ravel(), plane_eq([xyz[0].ravel(), xyz[1].ravel()], *popt)))

1.0


In [677]:
popt

array([ 0.00182323, -0.00273922,  0.46392538])

In [678]:
z0 = 500e-6 #* np.sin(np.radians(test.tth_arr))

In [665]:
def _find_intersection(x0, y0, z0, xyz, popt):
    A, B, C = popt
    a, b, c = xyz

    t = (z0 - C - A * x0 - B * y0) / (A * a + B * b - c)

    xf = x0 + t * a
    yf = y0 + t * b
    zf = z0 + t * c

    return np.asarray([xf, yf, zf])

In [679]:
out = _find_intersection(0, 0, z0, xyz, popt)

In [681]:
(xyz - out)**2

(3, 1944, 3072)

In [685]:
dist = np.sqrt((xyz[0] - out[0])**2 + (xyz[0] - out[0])**2 + (xyz[0] - out[0])**2)

In [688]:
test.plot_image(dist * 1e6)

In [565]:
x, y, z = out

# Must be arctan2 functions
chi = np.degrees(np.arctan2(y, x))
tth = np.degrees(np.arctan2(np.sqrt(x**2 + y**2), z))

# Convert original chi to new units???
base_chi = test.chi_arr.copy()


#chi_err = 

In [569]:
tth_2_d(0.011, wavelength=energy_2_wavelength(15e3))

4305.315943905888

In [566]:
test.plot_image(test.tth_arr - tth)

In [489]:
test.plot_image(test.chi_arr - chi)

In [551]:
test.plot_image(test.chi_arr)

In [550]:
test.plot_image(chi)

In [529]:
# No idea why this works or when it will fail
chi_img = 180 - chi.copy()
chi_img[chi_img > 180] -= 360
test.plot_image(chi_img)

In [512]:
#chi_img = test.chi_arr.copy()
#chi_img[chi_img < 0] += 360
#chi_img -= 180
#test.plot_image(chi_img)

In [494]:
x

array([[0.25172495, 0.25165045, 0.25157595, ..., 0.02259074, 0.02251591,
        0.02244109],
       [0.25172474, 0.25165024, 0.25157574, ..., 0.02259072, 0.0225159 ,
        0.02244107],
       [0.25172452, 0.25165002, 0.25157552, ..., 0.0225907 , 0.02251588,
        0.02244106],
       ...,
       [0.25129372, 0.25121935, 0.25114498, ..., 0.02255213, 0.02247743,
        0.02240273],
       [0.25129349, 0.25121912, 0.25114475, ..., 0.0225521 , 0.02247741,
        0.02240271],
       [0.25129326, 0.25121889, 0.25114452, ..., 0.02255208, 0.02247738,
        0.02240269]])

In [320]:
dwell = 1
det_sets = '_ln_1x1_'

dark_dir = '''D:\\Musterman_postdoc\\20240125_Musterman\\dark_fields\\fit_images\\'''
dark_paths = np.array(os.listdir(dark_dir))
offset_mask = [det_sets in x and 'offset' in x for x in dark_paths]
offset_df = io.imread(f'{dark_dir}{dark_paths[offset_mask][0]}').astype(np.float32)
scale_mask = [det_sets in x and 'slope' in x for x in dark_paths]
slope_df = io.imread(f'{dark_dir}{dark_paths[scale_mask][0]}').astype(np.float32)

dark_field = slope_df * dwell + offset_df
scaled_df = dark_field * np.median(test.map.min_image) / np.median(dark_field)
test.map.correct_dark_field(dark_field=dark_field)
#test.map.correct_dark_field(dark_field=scaled_df)

Correcting dark-field...done!


In [321]:
param = np.genfromtxt('''D:\\Musterman_postdoc\\20240125_Musterman\\amended\\scan152636_energy_rc_parameters.txt''')
energies, i0, im, it = param.reshape(param.shape[0], 1, param.shape[-1])

In [322]:
flat_dir = '''D:\\Musterman_postdoc\\20240125_Musterman\\flat_fields\\'''
flat_paths = np.array(os.listdir(flat_dir))
flat_mask = [det_sets in x for x in flat_paths]
flat_field = io.imread(f'{flat_dir}{flat_paths[flat_mask][0]}')
test.map.correct_flat_field(flat_field=flat_field)

Correcting flat-field...done!


In [323]:
test.map.normalize_scaler(scaler_arr=i0)

Normalize image scalers...done!


In [324]:
test.map.correct_outliers()

Finding and correcting image outliers...done!


In [349]:
cif_dir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Literature\\CIF\\'''
test.clear_phases()
test.load_phase('AMCSD\\Silicon_0011243.cif', filedir=cif_dir, phase_name="silicon1")

q_arrays = []
polarizations = []
lorentzs = []
solid_angles = []
absorptions = []

exp_dict = {
    'attenuation_length' : 0,
    'mode' : 'transmission',
    'thickness' : 500 # microns
}

for energy in energies.squeeze():
    test.energy = energy
    test._del_arr()
    exp_dict['attenuation_length'] = test.phases['silicon1'].absorption_length(en=energy * 1e3)

    test.map.apply_lorentz_correction(ai=test.ai, apply=False)
    test.map.apply_polarization_correction(ai=test.ai, apply=False)
    test.map.apply_solidangle_correction(ai=test.ai, apply=False)
    test.map.apply_absorption_correction(exp_dict=exp_dict, ai=test.ai, apply=False)

    q_arrays.append(test.q_arr)
    polarizations.append(test.map.polarization_correction)
    lorentzs.append(test.map.lorentz_correction)
    solid_angles.append(test.map.solidangle_correction)
    absorptions.append(test.map.absorption_correction)

In [350]:
for i, energy in enumerate(energies.squeeze()):
    test.map.images[0, i] /= polarizations[i]
    test.map.images[0, i] /= lorentzs[i]
    test.map.images[0, i] /= solid_angles[i]
    #test.map.images[0, i] /= absorptions[i]

In [262]:
test.map.estimate_background(method='bruckner', binning=8, min_prominence=0.01)
#test.map.remove_background()
#test.map.apply_defect_mask()

Estimating background with Bruckner algorithm.


100%|██████████| 31/31 [00:09<00:00,  3.21it/s]


In [240]:
test.plot_image([0, 0])

In [239]:
test.plot_image(test.map.background[0, 0])

In [263]:
test.map.remove_background()

Removing background...done!


In [309]:
#test.map.apply_defect_mask(min_bounds=(-5, 1e-4), max_bounds=(-1e-4, np.inf))
test.map.apply_defect_mask(min_bounds=(-5, 0.1), max_bounds=(-0.1, np.inf))
test.plot_image(test.map.defect_mask)

In [310]:
test.map.rescale_images(arr_min=0)
test.map.finalize_images()

Caution: Images not corrected for:
	pixel_distortions
	polar_calibration
	lorentz
	polarization
	solid_angle
	absorption
	background
Cleaning and updating image information...
Diffraction map size is 706.219 MB.
Compressing and writing images to disk.
This may take awhile...
done!


In [201]:
test.plot_interactive_map(img_vmin=0, img_vmax=0.01)

In [725]:
tth_2_d(test.tth_arr[371, 1916], wavelength=test.wavelength)

3.2941490739381143

In [524]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

im = ax.imshow(test.map.images[0, 13, 371 - 50:371 + 50, 1916-50:1916+50])
fig.colorbar(im, ax=ax)

plt.show()

In [311]:
q1 = q_arrays[13][:, 371, 1916]

q_image_list = []
for i in range(test.map.num_pixels):

    image = test.map.images[0, i]

    q_mask = np.ones_like(test.map.mask, dtype=np.bool_)
    scale = 0.025

    q = q_arrays[i]

    for i in range(len(q1)):
        mask = (((q1[i] - scale < q[i]))
                & (q[i] < (q1[i] + scale)))
        q_mask *= mask

    q_reduced = q[:, q_mask]
    int_reduced = image[q_mask]
    q_image = np.vstack([q_reduced, int_reduced])

    q_image_list.append(q_image)

In [217]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

for i in range(len(q_image_list)):
    int_mask = q_image_list[i][-1] > 0.1
    ax.scatter(*q_image_list[i][:3, int_mask],
               c=q_image_list[i][-1, int_mask], s=1, alpha=0.5)
    
    #ax.scatter(*q_image_list[i][:3],
    #           c=q_image_list[i][-1], s=1, alpha=0.5)

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')
plt.show()

In [312]:
from scipy.interpolate import griddata

all_qx = []
all_qy = []
all_qz = []
all_int = []

for q_image in q_image_list:
    all_qx.extend(q_image[0])
    all_qy.extend(q_image[1])
    all_qz.extend(q_image[2])
    all_int.extend(q_image[3])

# Find bounds
x_min = np.min(all_qx)
x_max = np.max(all_qx)
y_min = np.min(all_qy)
y_max = np.max(all_qy)
z_min = np.min(all_qz)
z_max = np.max(all_qz)

# Generate q-space grid
gridstep = 0.001
xx = np.linspace(x_min, x_max, int((x_max - x_min) / gridstep))
yy = np.linspace(y_min, y_max, int((y_max - y_min) / gridstep))
zz = np.linspace(z_min, z_max, int((z_max - z_min) / gridstep))

grid = np.array(np.meshgrid(xx, yy, zz))
grid = grid.reshape(3, grid.size // 3).T

points = np.array([all_qx, all_qy, all_qz]).T

int_grid = griddata(points, all_int, grid, method='nearest')
int_grid = int_grid.reshape(yy.shape[0], xx.shape[0], zz.shape[0]).T

In [588]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.hist(int_grid.ravel(), bins=1000)

plt.show()

In [222]:
from skimage import measure
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

spacing = 0.005

vol = int_grid
#vol = GaussianFunctions.func_3d(grid, *popt).reshape(*int_grid.shape)
#vol = GaussianFunctions.func_3d(np.array(np.meshgrid(xx, yy, zz)), *popt).T.reshape(*int_grid.shape)

iso_vals = np.linspace(np.min(int_grid), np.max(int_grid), 7)[1:-1]
iso_vals = [0.2, 0.4, 0.8, 1.6, 3.2, 6.4]

for i, iso_val in enumerate(iso_vals):

    #vol = int_grid
    #iso_val = 10000
    #spacing = 0.001
    color = cm.viridis(i / len(iso_vals))
    verts, faces, _, _ = measure.marching_cubes(vol, iso_val)

    # I have no idea how the axes got so mixed up...
    plot_x = np.interp(verts[:, 1], range(len(xx)), xx)
    plot_y = np.interp(verts[:, 2], range(len(yy)), yy)
    plot_z = np.interp(verts[:, 0], range(len(zz)), zz)

    ax.plot_trisurf(plot_x, plot_y, plot_z, triangles=faces, alpha=0.1, color=color)

ax.set_xlim(np.min(all_qx), np.max(all_qx))
ax.set_ylim(np.min(all_qy), np.max(all_qy))
ax.set_zlim(np.min(all_qz), np.max(all_qz))

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')
plt.show()

In [223]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

for i in range(len(q_image_list)):
    int_mask = q_image_list[i][-1] > 0.25
    ax.scatter(*q_image_list[i][:3, int_mask],
               c=q_image_list[i][-1, int_mask], s=1, alpha=0.5)
    
spacing = 0.0001
iso_val = 0.25
vol = int_grid

color = cm.viridis(i / len(iso_vals))
verts, faces, norms, vals = measure.marching_cubes(vol, iso_val, gradient_direction='ascent')

# I have no idea how the axes got so mixed up...
plot_x = np.interp(verts[:, 1], range(len(xx)), xx)
plot_y = np.interp(verts[:, 2], range(len(yy)), yy)
plot_z = np.interp(verts[:, 0], range(len(zz)), zz)

ax.plot_trisurf(plot_x, plot_y, plot_z, triangles=faces, alpha=0.1, color=color)

ax.set_xlim(np.min(all_qx), np.max(all_qx))
ax.set_ylim(np.min(all_qy), np.max(all_qy))
ax.set_zlim(np.min(all_qz), np.max(all_qz))

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')
plt.show()

In [313]:
p0 = [
    np.max(all_int), # amp
    all_qx[np.argmax(all_int)], # x0
    all_qy[np.argmax(all_int)], # y0
    all_qz[np.argmax(all_int)], # z0
    0.01, # sigma_x
    0.01, # sigma_y
    0.01, # sigma_z
    0, # theta
    0 # phi
]


bounds = np.asarray([
    [0, np.inf],
    [np.min(all_qx), np.max(all_qx)],
    [np.min(all_qy), np.max(all_qy)],
    [np.min(all_qz), np.max(all_qz)],
    [0.00001, 0.1],
    [0.00001, 0.1],
    [0.00001, 0.1],
    [-180, 180],
    [-180, 180]
]).T


popt, pcov = curve_fit(GaussianFunctions.func_3d, [all_qx, all_qy, all_qz], all_int - np.median(all_int), p0=p0, bounds=bounds)

residual = GaussianFunctions.func_3d([all_qx, all_qy, all_qz], *p0) - all_int + np.median(all_int)

In [314]:
guess_cen = arbitrary_center_or_mass(all_int, all_qx, all_qy, all_qz)
print(guess_cen)

(1.7437570749982063, -0.7320869098330195, -0.23682225021300435)


In [315]:
popt

array([  6.62331212,   1.7411221 ,  -0.73458852,  -0.23912069,
         0.01420482,   0.00443726,   0.00267783,  -5.38102234,
       177.42206685])

In [316]:
(popt[1:4] - guess_cen) / popt[1:4] * 100

array([-0.15133793,  0.34054637,  0.96120534])

In [317]:
convert_qd(np.linalg.norm(popt[1:4]))

3.2985871672300617

In [318]:
convert_qd(np.linalg.norm(guess_cen))

3.2965800150037756

In [152]:
cif_dir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Literature\\CIF\\'''
test.clear_phases()
test.load_phase('AMCSD\\Silicon_0011243.cif', filedir=cif_dir, phase_name="silicon1")
test.load_phase('AMCSD\\Silicon_0011244.cif', filedir=cif_dir, phase_name="silicon2")

In [158]:
test.phases['silicon1'].absorption_length(en=energies[0] * 1e3)

array([395.53718877, 399.60809804, 403.73322778, 407.91694584,
       412.1577696 , 416.46031201, 420.82052704, 425.09356809,
       429.42225435, 433.80995727, 438.25892226, 442.68836784,
       447.17825307, 451.72672102, 456.24974968, 460.83012228,
       465.47297755, 470.0851427 , 474.75859384, 479.49424678,
       484.20412378, 488.96893891, 493.7971105 , 498.59655687,
       503.45982554, 508.38465562, 513.2798462 , 518.23864753,
       523.25689288, 528.24757263, 533.3015334 ])

In [183]:
3.299 - 3.135

0.16400000000000015

In [184]:
0.164 / 3.135

0.05231259968102074

In [275]:
from skimage import measure
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

spacing = 0.005

vol = int_grid
#vol = GaussianFunctions.func_3d(grid, *popt).reshape(*int_grid.shape)
#vol = GaussianFunctions.func_3d(np.array(np.meshgrid(xx, yy, zz)), *popt).T.reshape(*int_grid.shape)

iso_vals = np.linspace(np.min(int_grid), np.max(int_grid), 7)[1:-1]
iso_vals = [0.2, 0.4, 0.8, 1.6, 3.2, 6.4]

for i, iso_val in enumerate(iso_vals):

    #vol = int_grid
    #iso_val = 10000
    #spacing = 0.001
    color = cm.viridis(i / len(iso_vals))
    verts, faces, _, _ = measure.marching_cubes(vol, iso_val)

    # I have no idea how the axes got so mixed up...
    plot_x = np.interp(verts[:, 1], range(len(xx)), xx)
    plot_y = np.interp(verts[:, 2], range(len(yy)), yy)
    plot_z = np.interp(verts[:, 0], range(len(zz)), zz)

    ax.plot_trisurf(plot_x, plot_y, plot_z, triangles=faces, alpha=0.1, color=color)

u = np.linspace(0, 2 * np.pi, 5000)
v = np.linspace(0, np.pi, 5000)

for energy in energies[0, ::6]:
    radius = 2 * np.pi / energy_2_wavelength(energy)

    x =  radius * np.outer(np.cos(u), np.sin(v))
    y = radius * np.outer(np.sin(u), np.sin(v))
    z = radius * np.outer(np.ones(np.size(u)), np.cos(v)) - radius

    mask = np.all([x > x_min, x < x_max, y > y_min, y < y_max, z > z_min, z < z_max], axis=0)
    x = x[mask]
    y = y[mask]
    z = z[mask]

    ax.plot_trisurf(x, y, z, alpha=0.25, color='k', label='Ewald sphere')


ax.set_xlim(np.min(all_qx), np.max(all_qx))
ax.set_ylim(np.min(all_qy), np.max(all_qy))
ax.set_zlim(np.min(all_qz), np.max(all_qz))

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')
plt.show()

In [278]:
from skimage import measure
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

spacing = 0.005

vol = int_grid
#vol = GaussianFunctions.func_3d(grid, *popt).reshape(*int_grid.shape)
vol = GaussianFunctions.func_3d(np.array(np.meshgrid(xx, yy, zz)), *popt).T.reshape(*int_grid.shape)

iso_vals = np.linspace(np.min(int_grid), np.max(int_grid), 7)[1:-1]
iso_vals = [0.2, 0.4, 0.8, 1.6, 3.2]

for i, iso_val in enumerate(iso_vals):

    color = cm.viridis(i / len(iso_vals))
    verts, faces, _, _ = measure.marching_cubes(vol, iso_val)

    # I have no idea how the axes got so mixed up...
    plot_x = np.interp(verts[:, 1], range(len(xx)), xx)
    plot_y = np.interp(verts[:, 2], range(len(yy)), yy)
    plot_z = np.interp(verts[:, 0], range(len(zz)), zz)

    ax.plot_trisurf(plot_x, plot_y, plot_z, triangles=faces, alpha=0.1, color=color)

vol = int_grid
for i, iso_val in enumerate(iso_vals):

    color = cm.viridis(i / len(iso_vals))
    verts, faces, _, _ = measure.marching_cubes(vol, iso_val)

    # I have no idea how the axes got so mixed up...
    plot_x = np.interp(verts[:, 1], range(len(xx)), xx)
    plot_y = np.interp(verts[:, 2], range(len(yy)), yy)
    plot_z = np.interp(verts[:, 0], range(len(zz)), zz)

    ax.plot_trisurf(plot_x, plot_y, plot_z, triangles=faces, alpha=0.1, color=color)

ax.set_xlim(np.min(all_qx), np.max(all_qx))
ax.set_ylim(np.min(all_qy), np.max(all_qy))
ax.set_zlim(np.min(all_qz), np.max(all_qz))

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')
plt.show()

In [277]:
iso_vals

[0.2, 0.4, 0.8, 1.6, 3.2, 6.4]

In [185]:
thickness = 500
test.plot_image(thickness * np.sin(np.radians(test.tth_arr)))

In [200]:
test.plot_image(test.tth_arr)

In [ ]:
XRDmap_list = []

filedir = '''D:\\Musterman_postdoc\\20231030_Musterman\\'''
otherdir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Data\\Condensed Datasets\\'''
energies = [15, 14.95, 14.9, 14.85, 14.8, 15.05, 15.10, 15.15, 15.2]

for index, scanid in enumerate(range(149235, 149243 + 1)):
    map_filename = f'scan{scanid}_xrd.tiff'

    test = XRDMap.from_image_stack(map_filename,
                                   wd=filedir,
                                   energy=energies[index],
                                   scanid=scanid)
    test.set_calibration('scan149256_dexela.poni')

    dark_field = io.imread(f'{otherdir}dexela_df_avg.tif')
    scaled_df = dark_field * np.median(test.map.min_image) / np.median(dark_field)
    test.map.correct_dark_field(dark_field=scaled_df)
    test.map.correct_outliers()
    test.map.normalize_scaler()

    test.map.apply_lorentz_correction(ai=test.ai)
    test.map.apply_polarization_correction(ai=test.ai)
    test.map.apply_solidangle_correction(ai=test.ai)

    #test.map.estimate_background(method='bruckner', iterations=100)
    test.map.estimate_background(method='gaussian', sigma=50)
    test.map.remove_background()

    test.calibrate_images()
    test.map.apply_defect_mask(lower=0.1, upper=-0.1)

    XRDmap_list.append(test)

In [315]:
for test in XRDmap_list:
    test.plot_image([6, 6])

In [7]:
test = XRDmap_list[0]
test.interactive_map()

In [330]:
index = (6, 6)

q_image_list = []
test = XRDmap_list[-1]
image = test.map.images[index]
indices = np.unravel_index(np.argmax(image), image.shape)
tth = test.tth[indices[1]]
chi = test.chi[len(test.chi) - indices[0]]
q1 = q_vect(tth, chi, wavelength=test.wavelength)

In [21]:
q_image_list[-1].shape

(4, 458)

In [23]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

#ax.plot(test.map.images[6, 6][q_mask])
ax.plot(q_image_list[1][-1])

plt.show()

In [24]:
image = np.copy(test.map.images[index])
np.median(image[test.map.mask])

-0.00033950523

In [25]:
index = (6, 6)

q_image_list = []
test = XRDmap_list[-1]
image = test.map.images[index]
indices = np.unravel_index(np.argmax(image), image.shape)
tth = test.tth[indices[1]]
chi = test.chi[len(test.chi) - indices[0]]
q1 = q_vect(tth, chi, wavelength=test.wavelength)

for test in XRDmap_list:
    image = np.copy(test.map.images[index])
    image /= np.abs(np.median(image[test.map.mask]))

    #tth = test.tth[indices[1]]
    #chi = test.chi[len(test.chi) - indices[0]]

    q_mask = np.ones_like(test.map.mask, dtype=np.bool_)
    scale = 0.025

    x_coords, y_coords = np.meshgrid(test.tth, test.chi[::-1])
    q = q_vect(x_coords, y_coords, wavelength=test.wavelength)


    for i in range(len(q1)):
        mask = (((q1[i] - scale < q[i]))
                & (q[i] < (q1[i] + scale)))
        q_mask *= mask

    q_reduced = q[:, q_mask]
    int_reduced = image[q_mask]
    q_image = np.vstack([q_reduced, int_reduced])

    q_image_list.append(q_image)

In [29]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

for i in range(len(q_image_list)):
    int_mask = q_image_list[i][-1] > -1000
    ax.scatter(*q_image_list[i][:3, int_mask],
               c=q_image_list[i][-1, int_mask], s=10, alpha=0.5)
    
    #ax.scatter(*q_image_list[i][:3],
    #           c=q_image_list[i][-1], s=1, alpha=0.5)


ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')
plt.show()

In [34]:
p0 = [
    np.max(all_int), # amp
    all_qx[np.argmax(all_int)], # x0
    all_qy[np.argmax(all_int)], # y0
    all_qz[np.argmax(all_int)], # z0
    0.01, # sigma_x
    0.01, # sigma_y
    0.01, # sigma_z
    0, # theta
    0 # phi
]


bounds = np.asarray([
    [0, np.inf],
    [np.min(all_qx), np.max(all_qx)],
    [np.min(all_qy), np.max(all_qy)],
    [np.min(all_qz), np.max(all_qz)],
    [0.00001, 0.1],
    [0.00001, 0.1],
    [0.00001, 0.1],
    [-180, 180],
    [-180, 180]
]).T


popt, pcov = curve_fit(GaussianFunctions.func_3d, [all_qx, all_qy, all_qz], all_int - np.median(all_int), p0=p0, bounds=bounds)

residual = GaussianFunctions.func_3d([all_qx, all_qy, all_qz], *p0) - all_int + np.median(all_int)

In [35]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.plot(all_int - np.median(all_int), label='data')
ax.plot(GaussianFunctions.func_3d([all_qx, all_qy, all_qz], *p0), label='guess')
ax.plot(GaussianFunctions.func_3d([all_qx, all_qy, all_qz], *popt), label='fit')

ax.legend()
plt.show()

In [36]:
popt

array([13819.0083968 ,    -2.3269767 ,     0.56574442,    -0.38709093,
           0.00549107,     0.0065496 ,     0.01477776,     8.03253441,
         -49.1507193 ])

In [39]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

ext = np.max(np.abs(residual))

#plot = 'residual'
#plot = 'data'
#plot = 'guess'
plot = 'fit'

if plot == 'residual':
    ax.scatter(all_qx, all_qy, all_qz, c=residual, s=5, alpha=0.5,
               cmap='bwr', vmin=-ext, vmax=ext)

if plot == 'guess':
    ax.scatter(all_qx, all_qy, all_qz, c=GaussianFunctions.func_3d([all_qx, all_qy, all_qz], *p0), s=5, alpha=0.5,
            cmap='viridis')
    
if plot == 'data':
        ax.scatter(all_qx, all_qy, all_qz, c=all_int, s=5, alpha=0.5,
            cmap='viridis')
        
if plot == 'fit':
    ax.scatter(all_qx, all_qy, all_qz, c=GaussianFunctions.func_3d([all_qx, all_qy, all_qz], *popt), s=5, alpha=0.5,
            cmap='viridis')


ax.scatter(popt[1], popt[2], popt[3], c='k', marker='*', s=10)


ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')
plt.show()

In [33]:
from scipy.interpolate import griddata

all_qx = []
all_qy = []
all_qz = []
all_int = []

for q_image in q_image_list:
    all_qx.extend(q_image[0])
    all_qy.extend(q_image[1])
    all_qz.extend(q_image[2])
    all_int.extend(q_image[3])

# Find bounds
x_min = np.min(all_qx)
x_max = np.max(all_qx)
y_min = np.min(all_qy)
y_max = np.max(all_qy)
z_min = np.min(all_qz)
z_max = np.max(all_qz)

# Generate q-space grid
gridstep = 0.001
xx = np.linspace(x_min, x_max, int((x_max - x_min) / gridstep))
yy = np.linspace(y_min, y_max, int((y_max - y_min) / gridstep))
zz = np.linspace(z_min, z_max, int((z_max - z_min) / gridstep))

grid = np.array(np.meshgrid(xx, yy, zz))
grid = grid.reshape(3, grid.size // 3).T

points = np.array([all_qx, all_qy, all_qz]).T

int_grid = griddata(points, all_int, grid, method='nearest')
int_grid = int_grid.reshape(yy.shape[0], xx.shape[0], zz.shape[0]).T

In [76]:
np.array(np.meshgrid(xx, yy, zz)).shape

(3, 49, 49, 30)

In [77]:
vol = GaussianFunctions.func_3d(np.array(np.meshgrid(xx, yy, zz)), *popt).T.reshape(*int_grid.shape)

In [81]:
from skimage import measure
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

spacing = 0.001

vol = int_grid
#vol = GaussianFunctions.func_3d(grid, *popt).reshape(*int_grid.shape)
#vol = GaussianFunctions.func_3d(np.array(np.meshgrid(xx, yy, zz)), *popt).T.reshape(*int_grid.shape)

iso_vals = [1000, 3000, 5000, 7000, 9000, 11000, 13000]

for i, iso_val in enumerate(iso_vals):

    #vol = int_grid
    #iso_val = 10000
    #spacing = 0.001
    color = cm.viridis(i / len(iso_vals))
    verts, faces, _, _ = measure.marching_cubes(vol, iso_val, spacing=(spacing,) * 3)

    ax.plot_trisurf(verts[:, 0], verts[:,1], faces, verts[:, 2], alpha=0.1, color=color)


ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')
plt.show()

In [32]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})


ax.scatter(*grid.T, c=int_grid.T.ravel(), s=5, alpha=0.5)


ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')
plt.show()